In [19]:
# Instalação e setup
%pip -q install google-genai google-adk

# Imports
import os, textwrap, warnings
from google.colab import userdata
from datetime import date
from IPython.display import display, Markdown
from google import genai
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search

warnings.filterwarnings("ignore")
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

# Funções auxiliares
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text + "\n"
    return final_response

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Agente 1 – Buscador (filtra eventos futuros)
def agente_buscador_oportunidades(perfil_usuario, data_de_hoje):
    buscador = Agent(
        name="agente_buscador_oportunidades",
        model=MODEL_ID,
        instruction="""
        Você é um buscador especializado em encontrar oportunidades de crescimento pessoal e profissional na internet. Sua resposta deverá ser amigável com utilização moderada de emojis.

        Utilize apenas uma frase em sua introdução. Evite excesso de texto.

        Com base no perfil do usuário e na data atual, use a ferramenta google_search para buscar **eventos, cursos, bolsas, mentorias ou vagas cujas datas de realização ou inscrição sejam HOJE ou no FUTURO**.

        Ignore completamente eventos já encerrados ou com inscrições vencidas. Priorize conteúdos com:
        - Data de realização futura
        - Prazos de inscrição ainda abertos
        - Indicação clara de 'Inscrições abertas', 'Participe agora', 'Vem aí', 'Em breve', 'Inscreva-se até XX/XX'

        Busque no máximo 5 oportunidades que sejam REAIS, ATUAIS e RELEVANTES para o perfil informado.
        """,
        description="Agente que busca oportunidades atuais e futuras",
        tools=[google_search]
    )
    entrada = f"Perfil do usuário: {perfil_usuario}\nHoje é: {data_de_hoje}. Busque apenas oportunidades com prazos abertos ou datas futuras."
    return call_agent(buscador, entrada)

# Agente 2 – Classificador (tabela organizada)
def agente_classificador(oportunidades_encontradas):
    classificador = Agent(
        name="agente_classificador",
        model=MODEL_ID,
        instruction="""
        Você é um analista de oportunidades.

        Classifique cada item da lista abaixo e organize em uma tabela com as colunas:
        Título | Tipo (curso, evento, vaga, etc.) | Área (ex: tecnologia, design, etc.) | Urgência (baixo, médio, alto) | Link

        Urgência:
        - Alto: prazos nos próximos 5 dias
        - Médio: prazos entre 6 e 15 dias
        - Baixo: mais de 15 dias ou sem prazo definido

        Deixe a apresentação clara, com texto direto e sem rodeios.
        """,
        description="Agente que organiza oportunidades em tabela"
    )
    return call_agent(classificador, f"Oportunidades encontradas:\n{oportunidades_encontradas}")

# Agente 3 – Planejador (dividido por semana)
def agente_planejador(perfil_usuario, oportunidades_classificadas):
    planejador = Agent(
        name="agente_planejador",
        model=MODEL_ID,
        instruction="""
        Você é um planejador estratégico.

        Com base no perfil do usuário e nas oportunidades listadas, crie um plano de ação para os próximos 30 dias, dividido por semanas.

        Para cada semana, indique:
        - Quais oportunidades focar
        - Quais ações devem ser feitas (ex: enviar inscrição, assistir evento, revisar portfólio)
        - Uma dica breve e motivadora

        Use listas organizadas por semana, com tom direto e encorajador.
        """,
        description="Agente que sugere plano de ação semanal"
    )
    entrada = f"Perfil do usuário: {perfil_usuario}\nOportunidades classificadas:\n{oportunidades_classificadas}"
    return call_agent(planejador, entrada)

# Agente 4 – Coach (minimalista e elegante)
def agente_coach(perfil_usuario, plano_acao):
    coach = Agent(
        name="agente_coach",
        model=MODEL_ID,
        instruction="""
        Você é um coach virtual com tom elegante, direto e inspirador.

        Apresente o plano de ação do usuário com leveza, encorajamento e clareza.
        Use emojis com moderação e apenas quando fizerem sentido. Evite excesso de texto.

        O objetivo é deixar o usuário animado e confiante para começar. Seja breve e impactante.
        """,
        description="Coach motivacional com estilo minimalista"
    )
    entrada = f"Perfil do usuário: {perfil_usuario}\nPlano de ação:\n{plano_acao}"
    return call_agent(coach, entrada)

# Execução do sistema completo
print("🚀 Bem-vindo ao OPORTU.AI – seu curador de oportunidades de crescimento!")
perfil_usuario = input("🧑‍🎓 Descreva seu perfil e interesses (ex: 'sou estudante de design e busco estágio e eventos'): ")
data_de_hoje = date.today().strftime("%d/%m/%Y")

if not perfil_usuario:
    print("❗ Você precisa descrever seu perfil.")
else:
    print("\n🔎 Buscando oportunidades...")
    oportunidades = agente_buscador_oportunidades(perfil_usuario, data_de_hoje)
    display(Markdown("### 🎯 Oportunidades encontradas:"))
    display(to_markdown(oportunidades))

    print("\n📂 Classificando oportunidades...")
    oportunidades_classificadas = agente_classificador(oportunidades)
    display(Markdown("### 🗂️ Oportunidades classificadas:"))
    display(to_markdown(oportunidades_classificadas))

    print("\n🗓️ Criando plano de ação...")
    plano = agente_planejador(perfil_usuario, oportunidades_classificadas)
    display(Markdown("### 🛠️ Plano de ação sugerido:"))
    display(to_markdown(plano))

    print("\n💬 Preparando mensagem motivacional...")
    mensagem_final = agente_coach(perfil_usuario, plano)
    display(Markdown("### 💡 Coach Virtual diz:"))
    display(to_markdown(mensagem_final))


🚀 Bem-vindo ao OPORTU.AI – seu curador de oportunidades de crescimento!
🧑‍🎓 Descreva seu perfil e interesses (ex: 'sou estudante de design e busco estágio e eventos'): redator

🔎 Buscando oportunidades...


### 🎯 Oportunidades encontradas:

> Olá! 👋 Posso te ajudar a encontrar algumas oportunidades de crescimento na área de redação. 😊
> 
> 
> Com certeza! Confira algumas oportunidades que encontrei para você:
> 
> 1.  **Content Experience 2025:** O maior evento de Marketing de Conteúdo do Brasil acontecerá nos dias 13 e 14 de maio de 2025. Ideal para redatores que buscam aprimorar suas habilidades e conhecer as últimas tendências do mercado. 
> 2.  **Vagas de Redator Home Office:** Encontre diversas vagas de emprego para redator em regime de home office no Indeed e Jooble. 
> 3.  **Curso de Redação para o ENEM:** O Instituto Federal de São Paulo (IFSP) oferece um curso gratuito de redação para o ENEM com inscrições abertas até 2 de abril de 2025.  As vagas são preenchidas por ordem de inscrição.
> 4.  **Curso O Texto Publicitário Criativo - técnicas para redação de textos no ambiente online e offline:** A ESPM oferece este curso com início em 24 de junho de 2025.
> 5.  **Vaga de Estágio na área de Marketing - Redator - 100% Remota:** A Real Seguro Viagem está com vaga de estágio para redator, com atuação 100% remota e bolsa de R$1.300,00. 
> 
> Espero que essas opções te ajudem a alavancar sua carreira! 😊
> 



📂 Classificando oportunidades...


### 🗂️ Oportunidades classificadas:

...

### 🛠️ Plano de ação sugerido:

> ## Plano de Ação Estratégico para Redator (Próximos 30 Dias)
> 
> Aqui está um plano de ação focado para você aproveitar ao máximo as oportunidades listadas, dividido por semanas. Vamos lá!
> 
> **Semana 1: Foco Total em Redação Educacional**
> 
> *   **Oportunidade:** Curso de Redação para o ENEM (IFSP)
> *   **Ações:**
>     *   [ ] Inscreva-se no curso imediatamente (Urgência Alta!).
>     *   [ ] Reserve um horário fixo na sua semana para se dedicar ao curso.
>     *   [ ] Comece a explorar materiais complementares sobre redação para o ENEM.
> *   **Dica:** "Acredite no poder da educação para abrir portas! Este curso pode ser o diferencial que você precisa."
> 
> **Semana 2: Explorando o Universo do Marketing de Conteúdo**
> 
> *   **Oportunidades:**
>     *   Content Experience 2025 (Evento)
>     *   Curso O Texto Publicitário Criativo (ESPM)
> *   **Ações:**
>     *   [ ] Verifique a programação e custo do evento Content Experience 2025. Avalie se vale a pena participar (lembre-se da urgência baixa).
>     *   [ ] Explore o conteúdo do curso da ESPM e veja se ele se encaixa nos seus objetivos de carreira.
>     *   [ ] Comece a pesquisar sobre as tendências de marketing de conteúdo para 2025.
> *   **Dica:** "Mantenha a mente aberta para novas áreas! O marketing de conteúdo é um campo vasto e cheio de oportunidades."
> 
> **Semana 3: Caça às Oportunidades de Redação**
> 
> *   **Oportunidades:**
>     *   Vagas de Redator Home Office (Indeed e Jooble)
>     *   Vaga de Estágio na área de Marketing - Redator (Real Seguro Viagem)
> *   **Ações:**
>     *   [ ] Dedique um tempo para pesquisar e se candidatar às vagas de redator home office no Indeed e Jooble.
>     *   [ ] Prepare um currículo e portfólio matadores, destacando suas habilidades de redação.
>     *   [ ] Se a vaga de estágio na Real Seguro Viagem te interessar, candidate-se também!
> *   **Dica:** "A persistência é a chave! Não desanime com os "nãos", o "sim" está mais perto do que você imagina."
> 
> **Semana 4: Refinando Habilidades e Planejando o Futuro**
> 
> *   **Oportunidades:** Todas!
> *   **Ações:**
>     *   [ ] Revise todo o conteúdo do curso de redação para o ENEM.
>     *   [ ] Avalie se os cursos/eventos de marketing de conteúdo ainda fazem sentido para você.
>     *   [ ] Continue buscando vagas e aprimorando seu currículo e portfólio.
>     *   [ ] Defina seus próximos objetivos de carreira como redator.
> *   **Dica:** "A jornada é contínua! Use esta semana para celebrar suas conquistas e traçar novos rumos."



💬 Preparando mensagem motivacional...


### 💡 Coach Virtual diz:

> Olá! ✨ Vejo que você tem um plano de ação bem estruturado para os próximos 30 dias. Incrível!
> 
> **🚀 Semanas 1-3:** Foco em educação e busca de oportunidades.
> **🎯 Semana 4:** Reflexão e planejamento futuro.
> 
> Este plano está super alinhado para impulsionar sua carreira como redator. Mantenha o foco, ajuste conforme necessário e celebre cada pequena vitória. Você está no caminho certo! ✍️💼
